In [10]:
!pip install requests sentence-transformers faiss-cpu numpy


In [11]:
import re
import requests
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [12]:
doc_id = "1o0x1b-kSmHBzSOG5jP2tTywRi0AdygsM3FOkozO4BCI"
export_url = f"https://docs.google.com/document/d/{doc_id}/export?format=txt"

print("📥 Загружаем базу знаний из Google Docs...")
response = requests.get(export_url)
response.raise_for_status()  # ошибка, если документ не публичный
text = response.content.decode('utf-8')

# Очистка от лишних переносов
text = re.sub(r'\n+', '\n\n', text).strip()

# Разбиваем на абзацы
paragraphs = [p.strip() for p in text.split("\n\n") if p.strip() and len(p.strip()) > 20]
print(f"✅ Загружено {len(paragraphs)} фрагментов.")

📥 Загружаем базу знаний из Google Docs...
✅ Загружено 5078 фрагментов.


In [13]:
model_name = "intfloat/multilingual-e5-small"
print(f"🧠 Загружаем модель: {model_name}")
model = SentenceTransformer(model_name)

# Кодируем документы с префиксом
print("🔍 Создаём эмбеддинги...")
passage_embeddings = model.encode(
    ["passage: " + p for p in paragraphs],
    normalize_embeddings=True,
    show_progress_bar=True
)

🧠 Загружаем модель: intfloat/multilingual-e5-small
🔍 Создаём эмбеддинги...


Batches:   0%|          | 0/159 [00:00<?, ?it/s]

In [14]:
dimension = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(np.array(passage_embeddings).astype("float32"))


In [15]:
questions = [
    'Можно ли устроиться в УИИ на удаленную работу после окончания обучения?',
    'Нет опыта в программировании. Я смогу участвовать в стажировке?',
    'У УИИ есть собственный фреймворк? ',
    'Покажи отзывы об обучении на курсе Data science и нейронные сети?',
    'С какими библиотеками по нейронным сетям работает УИИ?',
    'Есть ли курсы которые стоят 60 000 рублей? Скинуть стоимость курсов',
    'Самый крупный проект УИИ как аутсорсинговой компании',
    'Какие есть стажировки с нейронными сетями?',
    'Какая стоимость обучения в Университете искусственного интеллекта?',
    'Какая продолжительность программы обучения и какие формы обучения предлагаются?',
    'Кто куратор курса «Data science и нейронные сети»',
    'Могу ли я на стажировке выполнять проект для своего бизнеса?',
    'Какие есть стажировки с Object detection?',
    'Были ли стажировки в IT компаниях?',
    'Отправьте ссылки на вебинары УИИ, которые уже прошли',
    'Какие хакатоны выигрывали студенты УИИ?',
    'Можно ли получить какой-либо курс в подарок?',
    'Какой тариф обучения подойдет для опытного программиста? '
]

In [16]:
def search(query, k=2):
    query_emb = model.encode("query: " + query, normalize_embeddings=True)
    query_emb = np.array(query_emb).astype("float32").reshape(1, -1)
    scores, indices = index.search(query_emb, k)
    return [{"score": float(scores[0][i]), "text": paragraphs[indices[0][i]]} for i in range(k)]


In [17]:
for i, q in enumerate(questions, 1):
    print(f"\n❓ Вопрос {i}: {q}")
    results = search(q)
    for j, res in enumerate(results, 1):
        print(f"  📌 (score={res['score']:.3f}) {res['text'][:250]}...")


❓ Вопрос 1: Можно ли устроиться в УИИ на удаленную работу после окончания обучения?
  📌 (score=0.894) * Студентам, прошедшим курс УИИ, не нужно будет тратить время на дорогу до работы. Возможность работать удаленно сэкономит 1200 часов в год. Это возвратная инвестиция....
  📌 (score=0.891) Все курсы УИИ предоставляют достаточно знаний для дальнейшего трудоустройства. После прохождения обучения в УИИ, выпускники осваивают новую профессию и могут устраиваться на работу....

❓ Вопрос 2: Нет опыта в программировании. Я смогу участвовать в стажировке?
  📌 (score=0.881) 4. Возможность устроить к себе в штат разработчика со стажировки...
  📌 (score=0.881) 3. Работа в команде и взаимодействие с руководителями: программа позволяет студентам активно сотрудничать с руководителем команды, другими участниками команды и клиентами. Они могут наслаждаться совместной работой и обмениваться опытом с группой из 1...

❓ Вопрос 3: У УИИ есть собственный фреймворк? 
  📌 (score=0.914) В УИИ существует своя 